# Nomenclature 

Dans tout le notebook :
    $f$ désignera la fonction dont on cherche à tracer les lignes de niveau,$c \in \mathbb R $ la valeur de la ligne de niveau
    <br\> En pratique on travaillera avec $c=0$ avec le changement 
    \begin{array}{l|rcl}
     g : & \mathbb R^2 & \longrightarrow & \mathbb R \\
         & x & \longmapsto & f(x) - c\end{array}  
     <br\> les points appartiendront à l'espace de départ : $\mathbb R^2$.
     <br/> $eps$ désignera une petite quantité en deçà de laquelle $g$ est considérée comme étant nulle.


# Traçage des lignes de niveau

L'idée de l'algorithme est de travailler sur des subdivisions -des cases- du domaine considéré

Dans chaque case on suit un schéma bien précis:
    - Trouver une graine
    - Propager la ligne de niveau

 ##  Trouver une graine dans une case

Par "graine" on entend un point tel que l'on est sûrs que la fonction est nulle avec une grande précision ($eps$)

Pour cela, on va utiliser la **méthode de dichotomie** : on cherche des points $a$ et $b$ de la case tels que :

$g(a) \leq 0 \leq g(b)$ et on exhibe $x \in [a,b], |g(x)|<eps$ 

En pratique on prend pour $a$ et $b$ des points sur le contour de la case ayant la même abscisse ou la même ordonnée

![illustration](img/IMG_20191022_232213.jpg)

![find_seed_global](img/IMG_20191022_232257.jpg)

# Propager la ligne

Après avoir trouvé une graine $ a = (x_0,y_0) $ de la case on cherche à trouver $a$ proche de $b$ tel que $|g(b)|<eps$
<br/> On utilise alors le **théorème des fonctions implicites** dont l'application fait intervenir la **méthode de Newton de recherche d'un zéro**
<br/> Le but est d'exhibier $(b_n)_{n \in \mathbb N}$ suite de points telle que : 
<br/> $\forall n \in \mathbb N^* ; ||b_n - b_{n-1}|| \approx h , |g(b_n)|<eps$

## Description de l'algorithme :
    
### step 1
On se place en $a' = (x_0 + h,y_0)$
### step 2
On effectue la méthode de Newton sur la fonction $g$ sur le segment $[(x_0 + h, y_m),(x_0 + h, y_M)]$ avec $y_m$ et $y_M$ les bornes selon l'axe y de la case
### step 3
On itère tant que $x_0 + n*h$ ne sort pas de la case
### step 4,5,6
On fait de même avec $x_0 - n*h$, $y_0 + n*h$ et $y_0 - n*h$


![methode de Newton](img/IMG_20191022_233232.jpg)

In [ ]:
def Propagation(f, axis, x0, y0, eps1, eps2, c = 0):
    
    ''' Fonction de propagation de la ligne de niveau, axis donne l'axe 
    sur lequel on se déplace, eps1 le pas de déplacement, x0 et y0 les points de référence
    eps2 donne la précision voulue pour la méthode de Newton'''
    
    if axis!= 'x' and axis != 'y' : raise ValueError('Axis mal donné')
    
    def g(p,q) : return f(p,q) - c
    
    t1 = time.time()
    
    if axis == 'y' : 
        y = y0
        x = x0 + eps1
    
    if axis == 'x':
        x = x0
        y = y0 + eps1
    
    nb_derive = derive(g,axis,x0,y0)
    
    if axis == 'y':
        def psi(w):
            return w - 1/nb_derive*g(x,w)
        
        while abs(g(x,y))>eps2 and abs(y) < 10**6 and time.time()-t1 < 10**-3: ## Conditions d'arrêt restrictrices
            y = psi(y)
        
        if time.time() - t1 >= 10**-3 : return x,10**7 ## 10**7 car le code ensuite interprète cela comme un échec de la méthode

    if axis == 'x':
        def psi(w):
            return w - 1/nb_derive*g(w,y)
        
        while abs(g(x,y))>eps2 and abs(x) < 10**6 and time.time()-t1 < 10**-3: ## Conditions d'arrêt restrictrices
            x = psi(x)
        
        if time.time() - t1 >= 10**-3 : return 10**7,y ## 10**7 car le code ensuite interprète cela comme un échec de la méthode

    
    return x,y 

Des conditions d'arrêt très restrictices ont été mise dans la méthode de Newton afin de pallier à tout risque de disfonctionnement de l'algorithme
<br/> En effet la méthode de Newton est **très** efficace, ce qui implique une **grande régularité de la fonction à tester**, régularité qui n'est dans notre cas valable que très près du point de référence $(x_0,y_0)$

![cas problèmatiques](img/IMG_20191022_233756.jpg)

## Explication des cas problématiques

### Cas 1:
On a ici une dérivée partielle trop faible, ce qui nous fait sortir de la case. Cette erreur survient si $f(x0+h,y0)$ est trop grand devant 0 ou/et si $\partial_y f(x_0,y_0)$ est trop petit. Dans le premier cas comme dans le second, une des seules solutions est de réduire le pas $h$

### Cas 2:

La fonction n'est pas assez régulière au voisinage de $(x_0,y_0)$, ce qui occasione parfois des boucles infinies (cf exemple) On est alors obligé de réduire le pas $h$ pour espérer garder assez de régularité